# imod2warp

## Description
This script will help you move a selection of **tomograms and alignments** from *imod* to be used in *warp* for subtomogram averaging.

### The Sample Folder
The sample folder contains *stacks* and *alignments* of all tilt series from one microscope session. All stacks should be acquired using the same parameters (defocus may change) and have unique names. This script expects a folder structure like this:

    sampleFolder/
        TS_XX/
            imod/
                TS_XX.mrc
                TS_XX.mrc.mdoc
                taSolution.log
                TS_XX.rawtlt
                TS_XX.xf

### The Averaging Folder

I would recommend setting up a folder for your averaging project. It will contain all data from *warp*, *relion* and *m* in separate folders to keep track of files. For *warp*, one folder per session should be used to avoid naming confusion (e.g. TS_11 from two different sessions). The folder will look something like this:

    averagingProject/
        warp210712/ 
        warp210823/  
        warp210935/  
        relion/
        m/
    
Each *warp* folder contains all data required for warp (micrographs, alignment files from etomo, mdoc) in the following structure:

    ./warpXXXXX/
        imod/
            TS_XX.mrc/
                taSolution.log
                TS_XX.mrc.rawtlt
                TS_XX.mrc.xf
        mdoc/
            TS_XX.mrc.mdoc
        TS_XX.mrc
        
All output files (reconstructions, subtomograms, power spectra) will be saved here also. 

## Set up Target Folder

Enter a *target_folder* for the averaging project, eg. `'/mnt/ome/Projects7/Benedikt/04_SFB/03_Relion/mSFB-SLayer-average'`.  
Enter a *session ID* for this specific microscopy session, eg. `'210920'`

In [301]:
import os
import shutil
import glob

# Input
target_folder = '/home/Medalia/BWimmer/Benedikt/04_SFB/03_Relion/test'
session_ID = '210920'

# Prepare Folder Structure for Warp / Relion / M project.

os.chdir(target_folder)

if os.path.exists("relion"):
    print("Relion Folder Exists.")
else:
    os.mkdir("relion")
    print("Created Relion Folder.")
    
if os.path.exists("m"):
    print("M Folder Exists.")
else:
    os.mkdir("m")
    print("Created M Folder.")
    
if os.path.exists("warp_" + session_ID):
    print("Warp Folder with this ID already Exists. Are you sure you want to continue?")
else:
    os.mkdir("warp_" + session_ID)
    os.mkdir("warp_" + session_ID +"/imod")
    os.mkdir("warp_" + session_ID +"/mdoc")
    print("Made Warp Folder.")

Created Relion Folder.
Created M Folder.
Made Warp Folder.


## Read Sample Folder, select relevant TS
Enter the *sample_folder* which contains the microscopy data.  
Enter the *pattern* which returns all tilt series folders, eg. `'TS*'` to return TS_XX.  
Enter the *suffix* (if any) that is attached to the stack name, eg. for folder TS_01 and stack name TS_01-ali.mrc enter `'-ali'`.  
Enter the *selected tilt series* (folder names) that you would like to include in your averaging project, eg. `['TS_01','TS_03']`


In [302]:
# Input. File suffix can be used if folder and file root are different, e.g. '-ali' for TS_01
sample_folder = '/home/Medalia/BWimmer/Benedikt/04_SFB/01_Krios-data/210920-SFB-KriosI/210920-Exp10-mSFB'
pattern = 'TS*'
selected_ts = ['TS_23']
suffix = ''

# Read all TS Folders
os.chdir(sample_folder)
all_ts = glob.glob(pattern,recursive=False)
print("Found the following TS folders: \n" + ', '.join([str(elem) for elem in all_ts]) + "\n")
print("Selected the following TS folders: \n" + '\n'.join([str(elem) for elem in selected_ts]) + "\n")

# Check for raw stacks, mdoc files and alignment results for the selected TS
for elem in selected_ts:
      
    source_root = sample_folder + '/' + elem + '/imod/'
    
    if os.path.isfile(source_root + elem + suffix + '.mrc'):
        print("Stack found for " + elem)
    else: 
        print("Stack missing for " + elem)
    if os.path.isfile(source_root + elem + suffix + '.mrc.mdoc'):
        print("Metadata found for " + elem)
    else: 
        print("Metadata missing for " + elem)
    if os.path.isfile(elem + '/imod/taSolution.log'):
        print("taSolution found for " + elem)
    else: 
        print("taSolution missing for " + elem)
    if os.path.isfile(elem + '/imod/' + elem + suffix + '.rawtlt'):
        print("rawtlt found for " + elem)
    else: 
        print("rawtlt missing for " + elem)
    if os.path.isfile(elem + '/imod/' + elem + suffix + '.xf'):
        print("xf found for " + elem)
    else: 
        print("xf missing for " + elem)
    print("\n")

Found the following TS folders: 
TS_10, TS_11, TS_12, TS_03.mrc.mdoc~, TS_13, TS_07, TS_08, TS_14, TS_15, TS_16, TS_17, TS_18, TS_10.mrc.mdoc~, TS_22, TS_23, TS_24, TS_25, TS_26, TS_27, TS_28, TS_20, TS_29, TS_30, TS_31, TS_32, TS_01, TS_02, TS_03, TS_04, TS_05, TS_09

Selected the following TS folders: 
TS_23

Stack found for TS_23
Metadata found for TS_23
taSolution found for TS_23
rawtlt found for TS_23
xf found for TS_23




## Function: Make spoofed mdocs 
Takes input .mdoc as provided by SerialEM, adds the extracted tilt micrographs from a folder. Takes an input the sample folder, the Tilt series ID (eg. TS_12), under which the mdoc is found, and the folder to which individual tilt images hae been extracted. 

In [303]:
def tilts_to_mdoc(sample_folder,ts_id,target_warp):
    
    # Get List of all single tilts. 
    os.chdir(target_warp)
    filenames = list()
    filenames = glob.glob(ts_id + '*.mrc',recursive=False)
    filenames = sorted(filenames)
    
    # Copy mdoc to target directory before processing.   
    original = sample_folder + '/' + ts_id + '/imod/' + ts_id + '.mrc.mdoc'
    output = target_warp + '/mdoc/' + ts_id + '.mrc.mdoc'  
    
    # Open File and parse some info.
    
    mdoc_count = 0
    counter = 0      
    
    with open(original,'r') as orig:
        lines = [line for line in orig]
    
    out = open(output,'w')
        
    for line in lines:
        if line.startswith('[ZValue = '):
            mdoc_count = mdoc_count + 1
        if line.startswith('FilterSlitAndLoss'):
            out.write(line)
            out.write('SubFramePath = X:\spoofed\\' +  filenames[counter] + '\nNumSubFrames = 1\nFrameDosesAndNumber = 3.41 1\n')
            counter = counter + 1;
        else:
            out.write(line)
    
    # Sanity Check: Are there as many mdoc entries as single-tilt files?
    if len(filenames) == mdoc_count:
        print('For ' + ts_id + ', numbers of mdoc entries and tilt counts match.')
    else:
        print('Something is wrong with either the mdoc file or the extracted tilts; the numbers do not match')
       
    print('Created mdoc for ' + ts_id + '.\n')

## Copy all Files to relevant Target Subfolders

In [307]:
target_warp = target_folder + '/warp_' + session_ID + '/'

for elem in selected_ts:
    
    source_root = sample_folder + '/' + elem + '/imod/'    
    
    os.mkdir(target_warp + 'imod/' + elem + '.mrc')
    
    # Copy tilt alignments to imod/TS_XX
    shutil.copy2(source_root + elem + suffix + '.rawtlt',target_warp + 'imod/' + elem + '.mrc/' + elem + '.mrc.rawtlt')
    shutil.copy2(source_root + elem + suffix + '.xf',target_warp + 'imod/' + elem + '.mrc/' + elem + '.mrc.xf')
    shutil.copy2(source_root + 'taSolution.log',target_warp + 'imod/' + elem + '.mrc/taSolution.log')
        
    # Extract frames from raw stack to warp directory
    os.chdir(target_warp)
    os.system('newstack -split 0 -quiet -append mrc -in ' + source_root + elem + suffix + '.mrc ' + elem + '_slice_')     
    
    # Create adapted mdoc file
    tilts_to_mdoc(sample_folder,elem,target_warp)
    
    

For TS_23, numbers of mdoc entries and tilt counts match.
Created mdoc for TS_23.

